<a href="https://colab.research.google.com/github/ShachiniMekala/PoliticalCommentAnaysis/blob/main/Comments_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Background Work

In [17]:
from google.colab import drive
drive.mount('/content/gdrive')

KeyboardInterrupt: ignored

In [ ]:
import os
import string
import nltk
nltk.download('punkt')
from nltk import FreqDist
import pandas as pd

In [19]:
#read corpus into dataframe
df_comments = pd.read_csv("./Comments.csv") 

In [20]:
#select only positive comments
df_positive = df_comments[df_comments["Annotation"] == "positive"]
df_positive_comments = df_positive['Comment']

#select only negative comments
df_negative = df_comments[df_comments["Annotation"] == "negative"]
df_negative_comments = df_negative['Comment']

# Data Preprocessing

In [21]:
def preprocessing(input_str):
  output_str=''
  punc = "'"+"!"+"("+")"+"-"+"["+"]"+"{"+"}"+";"+":"+"\""+","+"<"+"+"+">"+"."+"/"+"?"+"@"+"#"+"$"+"%"+"^"+"&"+"*"+"_"+"~"+"\\"

  #remove punctuations in comment
  for char in input_str:
    if char not in punc:
        output_str = output_str + char

  output_str = output_str.strip()

  #annotate the begining and the end of the sentence
  output_str = "<" + output_str + ">"

  #convert the string into lower case and tokenize
  word_tokens = nltk.word_tokenize(output_str.lower())
  
  return word_tokens

# Corpus

In [22]:
unigram_list_p=[]
bigram_list_p=[]
unigram_list_n=[]
bigram_list_n=[]

#preprocess positive comments
for comment in df_positive_comments:
 positive_tokens = preprocessing(comment)
 #list the unigrams in the positive corpus
 unigram_list_p = unigram_list_p + list(positive_tokens)
 #list the bigrams in the positive corpus
 bigrams_p = nltk.bigrams(positive_tokens)
 bigram_list_p = bigram_list_p + list(bigrams_p)

#preprocess negative comments
for comment in df_negative_comments:
 negative_tokens = preprocessing(comment)
 #list the unigrams in the negative corpus
 unigram_list_n = unigram_list_n + list(negative_tokens)
 #list the bigrams in the negative corpus
 bigrams_n = nltk.bigrams(negative_tokens)
 bigram_list_n = bigram_list_n + list(bigrams_n)

In [23]:
#unique word count in positive corpus
unique_count_p = len(set(unigram_list_p))

#unique word count in positive corpus
unique_count_n = len(set(unigram_list_n))

In [24]:
#frequency distribution for positive unigrams and bigrams
unigram_freq_p = FreqDist(unigram_list_p)
bigram_freq_p = FreqDist(bigram_list_p)

#frequency distribution for negative unigrams and bigrams
unigram_freq_n = FreqDist(unigram_list_n)
bigram_freq_n = FreqDist(bigram_list_n)

# Input Comment

In [25]:
#preprocess input comment
input_comment = input()

#preprocess the input comment
input_tokens=preprocessing(input_comment)
input_bigrams = list(nltk.bigrams(input_tokens))

#word count in input comment
word_count_input = len(input_tokens)

jaya wewa


# Calculate Probability

In [27]:
# calculate the probability of being positive

probability_p = 1.0

for bigram in range(len(input_bigrams)):

  if input_bigrams[bigram] in bigram_freq_p:
    count_bigram = bigram_freq_p[input_bigrams[bigram]]
  else:
    count_bigram = 0 
 
  if input_bigrams[bigram][0] in unigram_freq_p:
    count_unigram = unigram_freq_p[input_bigrams[bigram][0]]
  else:
    count_unigram = 0

#laplace smoothing
  prob_after_smoothing = (count_bigram + 1) / (count_unigram + unique_count_p)

 #probability of being positive
  probability_p = probability_p * (prob_after_smoothing)

print('Probability of being POSITIVE : '+str(probability_p))


Probability of being POSITIVE : 1.1801382148373763e-07


In [28]:
# calculate the probability of being negative

probability_n = 1.0

for bigram in range(len(input_bigrams)):

  if input_bigrams[bigram] in bigram_freq_n:
    prob_bigram = bigram_freq_n[input_bigrams[bigram]]
  else:
    prob_bigram = 0 
 
  if input_bigrams[bigram][0] in unigram_freq_n:
    prob_unigram = unigram_freq_n[input_bigrams[bigram][0]]
  else:
    prob_unigram = 0

 #laplace smoothing
  prob_after_smoothing = (prob_bigram + 1) / (prob_unigram + unique_count_n)

 #probability of being positive
  probability_n = probability_n * (prob_after_smoothing)

print('Probability of being NEGATIVE : '+str(probability_n))

Probability of being NEGATIVE : 1.5509195983893698e-08


# Classification

In [29]:
if probability_p > probability_n:
 print("Input is a POSITIVE comment")
 perplexity=pow(probability_p, (-1 / word_count_input))
 print("Perplexity : "+str(perplexity))

else:
 if probability_p < probability_n:
  print("Input is a NEGATIVE comment")
  perplexity=pow(probability_n, (-1 / word_count_input))
  print("Perplexity : "+str(perplexity))
 else:
  print("Input comment is NEUTRAL")

Input is a POSITIVE comment
Perplexity : 53.953147012888586
